In [1]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
import configparser

In [2]:
config = configparser.ConfigParser()

In [3]:
config.read('mysql.config')

['mysql.config']

In [4]:
print(config['MYSQL']['host_name'])
print(config['MYSQL']['user_name'])
print(config['MYSQL']['user_password'])
print(config['MYSQL']['db_name'])

localhost
root
MUGL4Skd
conn_school_attendance


In [5]:
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

In [6]:
connection = create_db_connection(config['MYSQL']['host_name'],
                                  config['MYSQL']['user_name'],
                                  config['MYSQL']['user_password'],
                                  config['MYSQL']['db_name'])

MySQL Database connection successful


In [13]:
pd.read_sql_query("""
                  SELECT * 
                  FROM att_table
                  LIMIT 10;
                  """, con = connection)

,District code,District name,Category,Student group,2021-2022 student count - year to date,2021-2022 attendance rate - year to date,2020-2021 student count,2020-2021 attendance rate,2019-2020 student count,2019-2020 attendance rate,Reporting period,Date update
0,00000CT,Connecticut,,All Students,500285,0.9169,496092,0.9294,508346,0.9479,June 2022,07/22/2022
1,00000CT,Connecticut,Homelessness,Students Experiencing Homelessness,1814,0.8348,1735,0.8155,3916,0.8884,June 2022,07/22/2022
2,00000CT,Connecticut,Students With Disabilities,Students With Disabilities,78417,0.8899,76487,0.8946,80365,0.9277,June 2022,07/22/2022
3,00000CT,Connecticut,Free/Reduced Lunch,Free Meal Eligible,168984,0.8851,176225,0.8861,193706,0.9314,June 2022,07/22/2022
4,00000CT,Connecticut,Free/Reduced Lunch,Reduced Price Meal Eligible,29905,0.9184,30886,0.9299,27507,0.9518,June 2022,07/22/2022
5,00000CT,Connecticut,Free/Reduced Lunch,Free/Reduced Price Meal Eligible,198889,0.8901,207111,0.8927,221213,0.9340,June 2022,07/22/2022
6,00000CT,Connecticut,English Learners,English Learners,43571,0.8976,40619,0.8948,45413,0.9389,June 2022,07/22/2022
7,00000CT,Connecticut,Race/Ethnicity,All other races,48700,0.9314,47339,0.9483,47260,0.9559,June 2022,07/22/2022
8,00000CT,Connecticut,Race/Ethnicity,Black or African American,63099,0.8941,62267,0.8931,64200,0.9401,June 2022,07/22/2022
9,00000CT,Connecticut,Race/Ethnicity,Hispanic/Latino of any race,146298,0.8935,138260,0.8975,136953,0.9362,June 2022,07/22/2022


#### What are the different dates that this data was updated on?

In [9]:
pd.read_sql_query("""
                  SELECT DISTINCT `Date update`
                  FROM att_table;
                  """, con = connection)

,Date update
0,07/22/2022


#### What are the different Student group entries?

In [14]:
pd.read_sql_query("""
                  SELECT DISTINCT `Student group`
                  FROM att_table;
                  """, con = connection)

,Student group
0,All Students
1,Students Experiencing Homelessness
2,Students With Disabilities
3,Free Meal Eligible
4,Reduced Price Meal Eligible
5,Free/Reduced Price Meal Eligible
6,English Learners
7,All other races
8,Black or African American
9,Hispanic/Latino of any race


#### How many different districts are there?

In [16]:
pd.read_sql_query("""
                  SELECT COUNT(DISTINCT(`District name`)) AS num_of_disctricts
                  FROM att_table;
                  """, con = connection)

,num_of_disctricts
0,200


#### How many different categories are there?

In [17]:
pd.read_sql_query("""
                  SELECT COUNT(DISTINCT(Category)) AS num_of_categories
                  FROM att_table;
                  """, con = connection)

,num_of_categories
0,7


#### What are the distinct categories and how many times do they appear in the dataset?

In [23]:
pd.read_sql_query("""
                  SELECT DISTINCT Category, COUNT(*) AS count
                  FROM att_table
                  GROUP BY 1
                  ORDER BY 2 DESC;
                  """, con = connection)

,Category,count
0,Race/Ethnicity,566
1,Free/Reduced Lunch,477
2,High Needs,390
3,,200
4,Students With Disabilities,184
5,English Learners,112
6,Homelessness,13


#### Show all the districts where students are experiencing homelessness

In [24]:
pd.read_sql_query("""
                  SELECT *
                  FROM att_table
                  WHERE `Student group` = 'Students Experiencing Homelessness';
                  """, con = connection)

,District code,District name,Category,Student group,2021-2022 student count - year to date,2021-2022 attendance rate - year to date,2020-2021 student count,2020-2021 attendance rate,2019-2020 student count,2019-2020 attendance rate,Reporting period,Date update
0,00000CT,Connecticut,Homelessness,Students Experiencing Homelessness,1814,0.8348,1735,0.8155,3916,0.8884,June 2022,07/22/2022
1,0150011,Bridgeport School District,Homelessness,Students Experiencing Homelessness,123,0.8455,92,0.7791,107,0.8635,June 2022,07/22/2022
2,0170011,Bristol School District,Homelessness,Students Experiencing Homelessness,65,0.8904,57,0.8217,116,0.8913,June 2022,07/22/2022
3,0640011,Hartford School District,Homelessness,Students Experiencing Homelessness,107,0.7986,28,0.7847,437,0.8454,June 2022,07/22/2022
4,0830011,Middletown School District,Homelessness,Students Experiencing Homelessness,35,0.8319,29,0.7756,39,0.8761,June 2022,07/22/2022
5,0890011,New Britain School District,Homelessness,Students Experiencing Homelessness,81,0.8201,163,0.8229,259,0.8844,June 2022,07/22/2022
6,0930011,New Haven School District,Homelessness,Students Experiencing Homelessness,234,0.7573,173,0.7284,509,0.8838,June 2022,07/22/2022
7,0950011,New London School District,Homelessness,Students Experiencing Homelessness,184,0.8869,256,0.8345,301,0.9274,June 2022,07/22/2022
8,1030011,Norwalk School District,Homelessness,Students Experiencing Homelessness,33,0.8129,21,0.8667,25,0.8855,June 2022,07/22/2022
9,1350011,Stamford School District,Homelessness,Students Experiencing Homelessness,57,0.8306,27,0.8390,75,0.8648,June 2022,07/22/2022


#### Verify that there is an 'All Students' group for each district.

In [20]:
pd.read_sql_query("""
                  SELECT DISTINCT(COUNT(`District name`))
                  FROM att_table
                  WHERE `Student group` = 'All Students';
                  """, con = connection)

,(COUNT(`District name`))
0,200


#### What are the districts with the maximum and minimum attendance rates?

In [32]:
pd.read_sql_query("""
                  SELECT `District name`, `2021-2022 attendance rate - year to date`
                  FROM att_table
                  WHERE `Student group` = 'All Students' AND 
                  `2021-2022 attendance rate - year to date` = 
                      (SELECT MAX(`2021-2022 attendance rate - year to date`)
                       FROM att_table
                       WHERE `Student group` = 'All Students');
                  """, con = connection)

,District name,2021-2022 attendance rate - year to date
0,Wilton School District,0.9596


In [43]:
pd.read_sql_query("""
                  SELECT `District name`, `2021-2022 attendance rate - year to date`
                  FROM att_table
                  WHERE `Student group` = 'All Students' AND 
                  `2021-2022 attendance rate - year to date` = 
                      (SELECT MIN(`2021-2022 attendance rate - year to date`)
                       FROM att_table
                       WHERE `Student group` = 'All Students');
                  """, con = connection)

,District name,2021-2022 attendance rate - year to date
0,EdAdvance,0.7785


#### What districts are increasing overall attendance from 2019 to 2022?

In [44]:
pd.read_sql_query("""
                  SELECT COUNT(*) AS count
                  FROM (SELECT `District name`, `2021-2022 attendance rate - year to date`,
                               `2020-2021 attendance rate`, `2019-2020 attendance rate`
                        FROM att_table
                        WHERE `Student group` = 'All Students' AND 
                              `2021-2022 attendance rate - year to date` > `2020-2021 attendance rate` AND
                              `2020-2021 attendance rate` > `2019-2020 attendance rate`) increasing;
                  """, con = connection)

,count
0,0


#### How many districts have strictly decreasing attendance rates?

In [52]:
pd.read_sql_query("""
                  SELECT COUNT(*) AS count
                  FROM (SELECT `District name`, `2021-2022 attendance rate - year to date`,
                               `2020-2021 attendance rate`, `2019-2020 attendance rate`
                        FROM att_table
                        WHERE `Student group` = 'All Students' AND 
                              `2021-2022 attendance rate - year to date` < `2020-2021 attendance rate` AND
                              `2020-2021 attendance rate` < `2019-2020 attendance rate`) decreasing;
                  """, con = connection)

,count
0,80


#### What are they?

In [46]:
pd.read_sql_query("""
                  SELECT `District name`, `2021-2022 attendance rate - year to date`,
                         `2020-2021 attendance rate`, `2019-2020 attendance rate`
                  FROM att_table
                  WHERE `Student group` = 'All Students' AND 
                        `2021-2022 attendance rate - year to date` < `2020-2021 attendance rate` AND
                        `2020-2021 attendance rate` < `2019-2020 attendance rate`;
                  """, con = connection)

,District name,2021-2022 attendance rate - year to date,2020-2021 attendance rate,2019-2020 attendance rate
0,Connecticut,0.9169,0.9294,0.9479
1,Ashford School District,0.9309,0.9550,0.9585
2,Berlin School District,0.9341,0.9542,0.9562
3,Bethel School District,0.9375,0.9506,0.9563
4,Bloomfield School District,0.9293,0.9385,0.9532
...,...,...,...,...
75,Achievement First Hartford Academy District,0.8827,0.9051,0.9589
76,Elm City College Preparatory School District,0.8675,0.9354,0.9562
77,Capital Preparatory Harbor School District,0.9102,0.9399,0.9632
78,Connecticut Technical Education and Career System,0.8985,0.9321,0.9480


#### What districts have the highest percent homeless in 2021-2022?

In [54]:
pd.read_sql_query("""
                  SELECT a.`District name`, 
                         ((a.`2021-2022 student count - year to date`/b.`2021-2022 student count - year to date`) * 100)
                  FROM att_table a JOIN att_table b ON a.`District code` = b.`District code`
                  WHERE a.`Student group` = 'Students Experiencing Homelessness' AND
                        b.`Student group` = 'All Students'
                  GROUP BY 1, 2
                  ORDER BY 2 DESC
                  LIMIT 20;
                  """, con = connection)

,District name,((a.`2021-2022 student count - year to date`/b.`2021-2022 student count - year to date`) * 100)
0,New London School District,6.0229
1,Waterbury School District,1.3759
2,New Haven School District,1.2980
3,Bristol School District,0.8738
4,New Britain School District,0.8711
5,Middletown School District,0.8053
6,Bridgeport School District,0.6655
7,Hartford School District,0.6536
8,Capitol Region Education Council,0.5283
9,Connecticut,0.3626


#### What school districts have the worst attendance rate in 2021-2022?

In [55]:
pd.read_sql_query("""
                  SELECT `District name`, `2021-2022 attendance rate - year to date`
                  FROM att_table
                  WHERE `Student group` = 'All Students'
                  ORDER BY 2
                  LIMIT 12;
                  """, con = connection)

,District name,2021-2022 attendance rate - year to date
0,EdAdvance,0.7785
1,Eastern Connecticut Regional Educational Servi...,0.8273
2,Common Ground High School District,0.8321
3,New Haven School District,0.8383
4,Area Cooperative Educational Services,0.8500
5,Hartford School District,0.8564
6,Explorations District,0.8612
7,New Britain School District,0.8671
8,Elm City College Preparatory School District,0.8675
9,Windham School District,0.8724
